![ngo_project_image](ngo_project_image.jpg)

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.

Refer to the below ERD diagram for a visual representation of the relationships between these data tables:
<img src="erd.png" alt="ERD" width="50%" height="50%">


You will execute SQL queries to answer two questions, as listed in the instructions. Good luck!


List the top five assignments based on total value of donations, categorized by donor type. The output should include four columns: 1)_assignment_name_, 2) _region_, 3) _rounded_total_donation_amount_ rounded to two decimal places, and 4) _donor_type_, sorted by _rounded_total_donation_amount_ in descending order. 

In [5]:
-- highest_donation_assignments
SELECT 
	a.assignment_name,
	a.region,
	ROUND(SUM(dn.amount),2)AS rounded_total_donation_amount,
	dr.donor_type
FROM donations as dn
LEFT JOIN assignments as a
ON dn.assignment_id = a.assignment_id
LEFT JOIN donors as dr
ON dn.donor_id = dr.donor_id
GROUP BY a.assignment_name,a.region,dr.donor_type
ORDER BY rounded_total_donation_amount DESC
LIMIT 5;

,assignment_name,region,rounded_total_donation_amount,donor_type
0,Assignment_3033,East,3840.66,Individual
1,Assignment_300,West,3133.98,Organization
2,Assignment_4114,North,2778.57,Organization
3,Assignment_1765,West,2626.98,Organization
4,Assignment_268,East,2488.69,Individual


Identify the assignment with the highest impact score in each region, ensuring that each listed assignment has received at least one donation. The output should include four columns: 1) _assignment_name_, 2) _region_, 3) _impact_score_, and 4) _num_total_donations_, sorted by _region_ in ascending order. Include only the highest-scoring assignment per region, avoiding duplicates within the same region.

In [6]:
-- top_regional_impact_assignments
WITH d_count AS ( -- to filter for donation having at least one
    SELECT 
        a.assignment_id,
        COUNT(d.donation_id) as donation_count
    FROM assignments a
    LEFT JOIN donations d ON a.assignment_id = d.assignment_id
    GROUP BY a.assignment_id
    HAVING COUNT(d.donation_id) >= 1
),
row_num AS ( -- to include only highest impact score in each region
    SELECT 
        assignment_name,
        region,
        ROW_NUMBER() OVER(PARTITION BY region ORDER BY impact_score DESC) AS row_n
    FROM assignments
)
SELECT 
    row_num.assignment_name,
    a.region,
    a.impact_score,
    d_count.donation_count as num_total_donations
FROM assignments as a
LEFT JOIN d_count
    ON a.assignment_id = d_count.assignment_id
LEFT JOIN row_num
    ON a.assignment_name = row_num.assignment_name AND a.region = row_num.region
WHERE row_num.row_n = 1 -- only select highest impact_score
ORDER BY a.region;

,assignment_name,region,impact_score,num_total_donations
0,Assignment_316,East,10.00,2
1,Assignment_2253,North,9.99,1
2,Assignment_3547,South,10.00,1
3,Assignment_3764,West,9.99,1
